## Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os

os.chdir("..")

### Imports

In [4]:
from src.video_processing import PoseEstimator, VideoProcessor
from src.features import JointSeries, AngleSeries, FourierSeries
from src.models import kNNClassifier, LSTMClassifier

## Load model

In [5]:
import torch

file_path = os.path.join("models", "UCF101_knn_angles_model.pth")

knn = torch.load(file_path)

print(f"Loaded kNN model from {file_path}")

Loaded kNN model from models/UCF101_knn_angles_model.pth


## Record video via webcam

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have your knn class defined as KNN

# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(knn.train_data, knn.train_labels, test_size=0.2, random_state=42)

# Define a range of values for k and epsilon that you want to try
# k_values = [3, 5, 7, 9, 11, 13, 15]
k_values = [15]
epsilon_values = [1e-6, 1e-4, 1e-3, 1e-2, 1e-1]

best_accuracy = 0.0
best_k = None
best_epsilon = None

for k in k_values:
    for epsilon in epsilon_values:
        knn.k = k
        knn.metric.epsilon = epsilon
        knn.fit(train_data, train_labels)
        
        predictions = knn.predict(test_data)
        accuracy = accuracy_score(test_labels, predictions)
        
        print(f'k={k}, epsilon={epsilon}, Accuracy: {accuracy:.4f}')
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_k = k
            best_epsilon = epsilon

print(f'Best accuracy: {best_accuracy:.4f} with k={best_k} and epsilon={best_epsilon}')


2023-08-28 21:47:17,386 - INFO - Fitting kNN classifier to training data: 448 sample(s), 5 unique labels
2023-08-28 21:47:17,386 - INFO - Predicting class labels for input data: 113 sample(s)
2023-08-28 21:51:54,101 - INFO - Fitting kNN classifier to training data: 448 sample(s), 5 unique labels
2023-08-28 21:51:54,101 - INFO - Predicting class labels for input data: 113 sample(s)


k=15, epsilon=1e-06, Accuracy: 0.2743


2023-08-28 21:56:29,073 - INFO - Fitting kNN classifier to training data: 448 sample(s), 5 unique labels
2023-08-28 21:56:29,073 - INFO - Predicting class labels for input data: 113 sample(s)


k=15, epsilon=0.0001, Accuracy: 0.2743


2023-08-28 22:01:06,191 - INFO - Fitting kNN classifier to training data: 448 sample(s), 5 unique labels
2023-08-28 22:01:06,192 - INFO - Predicting class labels for input data: 113 sample(s)


k=15, epsilon=0.001, Accuracy: 0.2743


2023-08-28 22:05:43,051 - INFO - Fitting kNN classifier to training data: 448 sample(s), 5 unique labels
2023-08-28 22:05:43,052 - INFO - Predicting class labels for input data: 113 sample(s)


k=15, epsilon=0.01, Accuracy: 0.2743
k=15, epsilon=0.1, Accuracy: 0.2743
Best accuracy: 0.2743 with k=15 and epsilon=1e-06


In [ ]:
pose_estimator = PoseEstimator(
    model_complexity=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

VIDEO_INPUT_WEBCAM = 1

video_processor = VideoProcessor(
    pose_estimator=pose_estimator,
    webcam=VIDEO_INPUT_WEBCAM,
)

video_processor.process_video(
    show=True,
    height=800,
    width=None,
)

## Extract features

In [ ]:
joint_series = JointSeries(landmarks_series=video_processor.landmarks_series, fps=video_processor.fps)
angle_series = AngleSeries(joint_series=joint_series)
fourier_series = FourierSeries(angle_series=angle_series)


## Model prediction

In [ ]:
knn.predict(angle_series)

In [ ]:
lstm = LSTMClassifier(
    num_classes=len(set(y_train)),
    input_size=len(X_train[0].columns),
    hidden_size=32,
    num_layers=2,
    num_epochs=10,
    batch_size=8,
    learning_rate=0.001,
)